In [ ]:
import numpy as np
# Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [ ]:

# Load the dataset
df = pd.read_excel('DQN1 Dataset.xlsx')
# Take the first look at the dataset
df.info()

In [ ]:

# Prepare data for model
X = df.drop('healthRiskScore', axis=1)
y = df['healthRiskScore']
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=7)  # 30% test, 70% training

In [ ]:

# Create the first model
lr_model = LinearRegression()
# Fit the model
lr_model.fit(X_train, y_train)
# Predict using the model
y_pre_lr = lr_model.predict(X_test)


In [ ]:
# B1. First Optimization technique using 
# Stochastic Gradient Descent with StandardScaler for supporting
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', SGDRegressor(loss='squared_error', learning_rate='adaptive'))
])
# Fit the pipeline to the data
pipe.fit(X_train, y_train)
# Predict using the model
y_pre_SGD = pipe.predict(X_test)


In [ ]:
# B1. Second Optimization technique using 
# Huber Loss with StandardScaler for supporting
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import HuberRegressor
# Create pipeline to house scaler and estimator
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', HuberRegressor(epsilon=1.35, max_iter= 10000))
])
# Fit the pipeline to the data
pipe.fit(X_train, y_train)
# Predict using the model
y_pre_HR = pipe.predict(X_test)


In [ ]:
# B.2 Apply L2 Regularization using Ridge Regression
from sklearn.linear_model import Ridge

# Creat Ridge model
ridge_model = Ridge(alpha=1.0) 

# Fit the model
ridge_model.fit(X_train, y_train)

# Make predictions
y_pred_ridge = ridge_model.predict(X_test)

In [ ]:
# B.2 Apply L1 Regularization using Lasso Regression
from sklearn.linear_model import Lasso

# Create the Lasso model
lasso_model = Lasso(alpha=0.0001, max_iter=7000000) 

# Fit the model
lasso_model.fit(X_train, y_train)

# Make predictions
y_pred_lasso = lasso_model.predict(X_test)

In [ ]:
# B.3. Apply ensemble technique Voting 
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import VotingRegressor

# Define individual models
ridge = Ridge(alpha=5.0)
lasso = Lasso(alpha=0.0001, max_iter=7000000)
linear_model = LinearRegression()

# Ensemble using Voting (averaging predictions)
voting_reg = VotingRegressor([
    ('ridge', ridge),
    ('lasso', lasso)
    , ('linear', linear_model)
])
voting_reg.fit(X_train, y_train)

# Evaluate
y_pred_voting = voting_reg.predict(X_test)

In [ ]:
from sklearn.ensemble import BaggingRegressor

# Base model
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('linear', HuberRegressor(epsilon=1.35, max_iter= 10000))
])

# Bagging ensemble (100 Ridge models on random subsets)
bagging_reg = BaggingRegressor(
    estimator=pipe,
    n_estimators=10,
    max_samples=0.7,  # 80% of data per model
    bootstrap=True,   # Sample with replacement
    random_state=13
)
bagging_reg.fit(X_train, y_train)

# Evaluate
y_pred_bagging = bagging_reg.predict(X_test)
print(f'R2 = {r2_score(y_test, y_pred_bagging):.7f}')
print(f'Mean Squared Error = {mean_squared_error(y_test, y_pred_bagging):.4f}')
print(f'Mean Absolute Error = {mean_absolute_error(y_test, y_pred_bagging):.4f}')